# Hit Detection Notebook

This notebook demonstrates how to find the vessels and times that a particular satellite was able to see.

## Pre-requisites

This notebook presumes that data processing and precompute has already been completed. Please see the **Data Preparation** notebook.

Required libraries for this notebook (not the pre-computation one):

  * Numpy 1.19.2
  * Pandas v1.1.5
  * Numba 0.51.2
  * Holoviews 1.14
  * Bokeh 2.2.3
  * Datashader 0.11.1
  * PyTables 3.6.1
  * Jupyter (core=4.7, client=6.1.7)

These should be installed via `conda`, from the default Anaconda repository (not Conda-Forge).

The `skyfield` library should be installed via `pip`.  It is very important that `skyfield` version is 1.35, and the `sgp4` library that it installs is v2.14.

## Data and Locations

The required data variables are described below:

| Variable | Description | S3 location | 
| --- | :- | :- |
| `AIS_DIR` | Location of ais_????.h5 files or ais_????.interp.h5 files | `s3://anaconda-hit-finder-prod/AIS` or `s3://anaconda-hit-finder-prod/AIS_interp` |
| `SAT_DIR` | Location of precomputed satellite tracks. | `s3://anaconda-hit-finder-prod/satellites_active` or `s3://anaconda-hit-finder-prod/satellites_all` | 

By default, `AIS_DIR` is set to the current working directory (CWD) of this noteboook, and `SAT_DIR` looks for a directory called `satellites/` in the current directory.

## A comment on hardware

This algorithm is heavily parallelized and can take advantage of all cores on the machine.

Development was done on a Macbook Pro with 4 cores and 16 GB of memory, and Amazon AWS EC2 instances of type `m5zn.6xlarge` and `t3.2xlarge`. The target deployment environment is a workstation-grade 8 or 16 core machine with 16GB of memory.


In [ ]:
# Initial setup
# Make the notebook wider
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [ ]:
# Imports

import os
import pandas as pd


In [ ]:
# Set some configuration variables
# In general, these should be explicit paths with no variables or homedir (~)
AIS_DIR = "data/vessel data/Cleaned AIS"
SAT_DIR = "data/satellite data/index_active"

if not os.path.isdir(AIS_DIR) or not os.path.isdir(SAT_DIR):
    raise IOError("Invalid source data directory")

# Step 0. Configure the input parameters

In [ ]:
# The satellite we're interested in
norad_id = 25544  # The International Space Station

# The start and end times we're interested in.  For the sake of simplicity in
# this notebook, we are restricting to just one year.  The Python script
# is able to query multiple years.
start_time = pd.Timestamp("2014-12-31T00:00:01")
end_time = pd.Timestamp("2015-02-01T00:00:00")

# Based on the year of interest, also define the AIS file to look at
AIS_FILENAME = "ais_2015.h5"



# Step 1. Load the satellite data

In [ ]:
from scripts.sathelpers import SatelliteDataStore
satdata = SatelliteDataStore(SAT_DIR)

(times, lats, lons, alts) = satdata.get_precomputed_tracks(norad_id, start=start_time,
        end=end_time)
# The longitudes in the pre-computed satellite tracks range from 0-360,
# but we need them in (-180,180) format.
mask = lons > 180.0
lons[mask] -= 360

# Now convert to a dict that can by passed in to the intersection calculation
sat = pd.DataFrame({"date_time": times.astype("<M8[s]"),
       "lat": lats, "lon": lons, "alt": alts})

# Step 2. Load the AIS data

Since the example in this notebook is from the period of time of 2009, we just need to load its AIS tracks.

In [ ]:
ais = pd.read_hdf(os.path.join(AIS_DIR, AIS_FILENAME))
ais.sort_values(by="date_time", inplace=True)
ais.info()

# Step 3. Compute the visible points

In [ ]:
from scripts import intersect; intersect.PRINT_INFO=True

In [ ]:
reload(intersect)

In [ ]:
hits = intersect.compute_hits(sat, ais, start_time="2015-01-01", end_time="2015-01-17", workers=4)

In [ ]:
len(hits)

In [ ]:
hits2 = intersect.compute_hits(sat, ais, start_time="2015-01-01", end_time="2015-01-17", workers=4, assume_half_earth=True)

In [ ]:
len(hits2)

# Step 4. Visualize the results

In [ ]:
import panel as pn
import datetime as dt
import holoviews as hv
from colorcet import fire
from holoviews.operation.datashader import rasterize
hv.extension('bokeh')

In [ ]:
import plot_helpers

In [ ]:
t = plot_helpers.plot_points(hits)
#t2 = plot_helpers.plot_points(hits2)

In [ ]:
pn.Row(t).servable()